In [ ]:
import json
import requests
import html5lib
import bs4
from bs4 import BeautifulSoup
import scipy
import gensim
import gensim.downloader as api

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
tour_urls = ['https://www.explore.co.uk/holidays/machu-picchu-trek',
             'https://www.explore.co.uk/holidays/vietnam-historic-tour',
             'https://www.explore.co.uk/holidays/cycling-holiday-morocco']

In [ ]:
def return_response(sim_func, query, corpus):
    similarities = []
    for doc in corpus:
        similarity = sim_func(user_input, doc)
        print(similarity)
        similarities.append(similarity)
        tour_id = similarities.index(max(similarities))
    return tour_id, corpus_of_documents[similarities.index(max(similarities))]

In [ ]:
contents = []
tours = []
for index, url in enumerate(tour_urls):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html5lib')

    tours.append(soup.title.get_text().replace('\n', ' ').replace('\t', ''))
    table = soup.find('div', attrs = {'id':'itinerary'})

    itinerary = []  # a list to store quotes
    for row in table.findAll('div', attrs = {'class':'pr-i-desc'}): 
        itinerary.append(row.p.get_text())

    contents.append(itinerary)

In [ ]:
corpus_of_documents = []
for item in contents:
    full_details = ' '.join(item)
    corpus_of_documents.append(full_details)

In [ ]:
text_1 = "I'd like to visit South America and visit Machu Picchu"

In [ ]:
gen_docs = [[w.lower() for w in word_tokenize(text)]
        for text in corpus_of_documents]

In [ ]:
dictionary = gensim.corpora.Dictionary(gen_docs)

In [ ]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]

In [ ]:
tf_idf = gensim.models.TfidfModel(corpus)

In [ ]:
sims = gensim.similarities.Similarity('workdir/',tf_idf[corpus],
                                        num_features=len(dictionary))

In [ ]:
query_doc = [w.lower() for w in word_tokenize(text_1)]
query_doc_bow = dictionary.doc2bow(query_doc)

In [ ]:
# perform a similarity query against the corpus
query_doc_tf_idf = tf_idf[query_doc_bow]
# print(document_number, document_similarity)
print('Comparing Result:', sims[query_doc_tf_idf]) 

In [ ]:
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]